## Abrir dataset Coswara

In [14]:
import pandas as pd

# Cargar el CSV con la codificación 'ISO-8859-1'
csv_path = 'CoronaHack-Respiratory-Sound-Dataset/Corona-Hack-Respiratory-Sound-Metadata.csv'  # Cambia a la ruta correcta de tu archivo CSV
data = pd.read_csv(csv_path, encoding='ISO-8859-1')

# Mostrar las primeras filas para verificar la carga correcta
print(data.head())


                        USER_ID COUNTRY  AGE COVID_STATUS ENGLISH_PROFICIENCY  \
0  vK2bLRNzllXNeyOMudnNSL5cfpG2   India   24      healthy                   Y   
1  bjA2KpSxneNskrLBeqi4bqoTDQl2   India   72      healthy                   Y   
2  FSzobvJqOXf0rI6X05cHqOiU9Mu2   India   54      healthy                   Y   
3  EqDWckxbsETyHUeBLQ8jLtxlhir2   India   31      healthy                   Y   
4  FGRDO4IBbAejR0WHD5YbkXTCasg2   India   26      healthy                   Y   

  GENDER COUNTY_RO_STATE CITY_LOCALITY  Diabetes  Asthma  ...     DATES  \
0      M       Karnataka     Bangalore         0       0  ...  20200413   
1      M     Maharashtra         Thane         0       0  ...  20200413   
2      M     Maharashtra    Thane West         0       0  ...  20200413   
3      M       Karnataka     Bangalore         0       0  ...  20200413   
4      M         Haryana       gurgaon         0       0  ...  20200413   

                                      breathing-deep  \
0  /da

## Filtrar por claves necesarias

In [15]:
columnas_necesarias = [
    'USER_ID', 'AGE', 'GENDER', 'COVID_STATUS', 'counting-fast'
]

# Eliminar todas las demás columnas excepto las que están en 'columnas_necesarias'
data_filtrada = data[columnas_necesarias]

# Mostrar las primeras filas para verificar
print(data_filtrada.head())


                        USER_ID  AGE GENDER COVID_STATUS  \
0  vK2bLRNzllXNeyOMudnNSL5cfpG2   24      M      healthy   
1  bjA2KpSxneNskrLBeqi4bqoTDQl2   72      M      healthy   
2  FSzobvJqOXf0rI6X05cHqOiU9Mu2   54      M      healthy   
3  EqDWckxbsETyHUeBLQ8jLtxlhir2   31      M      healthy   
4  FGRDO4IBbAejR0WHD5YbkXTCasg2   26      M      healthy   

                                       counting-fast  
0  /data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...  
1  /data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...  
2  /data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...  
3  /data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...  
4  /data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...  


## Cambiar etiquetas a claves numericas

In [16]:
# Crear un diccionario de mapeo
etiquetas = {'healthy': 0,
            'recovered_full': 0, 
            'positive_asymp': 1, 
            'positive_mild': 1, 
            'positive_moderate': 1,
            'resp_illness_not_identified':1,
            'no_resp_illness_exposed':1}

# Mapear las etiquetas
data_filtrada['COVID_STATUS'] = data_filtrada['COVID_STATUS'].map(etiquetas).fillna(0).astype(int)

C:\Users\Vicen\AppData\Local\Temp\ipykernel_23036\3040699067.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtrada['COVID_STATUS'] = data_filtrada['COVID_STATUS'].map(etiquetas).fillna(0).astype(int)


## Guardar nuevo dataframe

In [17]:
# Guardar el DataFrame filtrado en un nuevo archivo CSV

output_csv_path = 'nuevo_dataset.csv'  # Cambia a la ruta y nombre de tu elección
data_filtrada.to_csv(output_csv_path, index=False)


## Pasar los audios a espectrogramas

In [21]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np  # Asegúrate de importar numpy

# Cargar el dataset filtrado (ajusta la ruta según sea necesario)
csv_path = 'nuevo_dataset.csv'
data_filtrada = pd.read_csv(csv_path)

# Directorio de salida para guardar los espectrogramas
output_dir = 'espectrogramas'
os.makedirs(output_dir, exist_ok=True)

# Función para generar espectrogramas
def generar_espectrograma(audio_path, output_path):
    if not os.path.isfile(audio_path):
        print(f"Archivo no encontrado: {audio_path}")
        return  # Salir si el archivo no existe

    try:
        y, sr = librosa.load(audio_path, sr=None)  # Mantener la frecuencia de muestreo original
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
        S_dB = librosa.power_to_db(S, ref=np.max)

        # Guardar el espectrograma como imagen
        plt.figure(figsize=(2, 2))
        librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel')
        plt.axis('off')
        plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
        plt.close()
    except Exception as e:
        print(f"Error procesando {audio_path}: {e}")

# Recorrer cada fila del DataFrame y procesar cada columna de audio
for index, row in data_filtrada.iterrows():
    user_id = row['USER_ID']
    covid_status = row['COVID_STATUS']
    
    # Procesar las columnas de audio específicas
    for col in ['counting-fast']:
        
        # Ruta de audio
        audio_path = row[col]
        if pd.notna(audio_path):  # Verificar que el path no sea NaN
            # Convertir la ruta de Unix a Windows (si es necesario)
            audio_path = "CoronaHack-Respiratory-Sound-Dataset" + audio_path
            audio_path = os.path.abspath(audio_path)  # Convertir a ruta absoluta
            print(f"Intentando cargar el archivo: {audio_path}")  # Para depuración
            
            # Crear nombre único para cada espectrograma
            output_path = os.path.join(output_dir, f"{user_id}_{covid_status}_{col}.png")
            generar_espectrograma(audio_path, output_path)
            print(f"Espectrograma guardado en: {output_path}")


Intentando cargar el archivo: c:\Users\Vicen\OneDrive\Escritorio\Covid\DeteccionCovidRespiracion\CoronaHack-Respiratory-Sound-Dataset\data\train\20200413\vK2bLRNzllXNeyOMudnNSL5cfpG2\counting-fast.wav
Espectrograma guardado en: espectrogramas\vK2bLRNzllXNeyOMudnNSL5cfpG2_0_counting-fast.png
Intentando cargar el archivo: c:\Users\Vicen\OneDrive\Escritorio\Covid\DeteccionCovidRespiracion\CoronaHack-Respiratory-Sound-Dataset\data\train\20200413\bjA2KpSxneNskrLBeqi4bqoTDQl2\counting-fast.wav
Espectrograma guardado en: espectrogramas\bjA2KpSxneNskrLBeqi4bqoTDQl2_0_counting-fast.png
Intentando cargar el archivo: c:\Users\Vicen\OneDrive\Escritorio\Covid\DeteccionCovidRespiracion\CoronaHack-Respiratory-Sound-Dataset\data\train\20200413\FSzobvJqOXf0rI6X05cHqOiU9Mu2\counting-fast.wav
Espectrograma guardado en: espectrogramas\FSzobvJqOXf0rI6X05cHqOiU9Mu2_0_counting-fast.png
Intentando cargar el archivo: c:\Users\Vicen\OneDrive\Escritorio\Covid\DeteccionCovidRespiracion\CoronaHack-Respiratory-Soun

c:\Users\Vicen\anaconda3\envs\audio_env\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Espectrograma guardado en: espectrogramas\nMIOAh7qRFf3pqbchclOLKbPDOm1_0_counting-fast.png
Intentando cargar el archivo: c:\Users\Vicen\OneDrive\Escritorio\Covid\DeteccionCovidRespiracion\CoronaHack-Respiratory-Sound-Dataset\data\train\20200413\Pr79fov0a5TW5KPo0XACvIqMtO72\counting-fast.wav
Espectrograma guardado en: espectrogramas\Pr79fov0a5TW5KPo0XACvIqMtO72_0_counting-fast.png
Intentando cargar el archivo: c:\Users\Vicen\OneDrive\Escritorio\Covid\DeteccionCovidRespiracion\CoronaHack-Respiratory-Sound-Dataset\data\train\20200413\60Ig4PmU9NMqKGDMR8Um0Tc2KXB3\counting-fast.wav
Espectrograma guardado en: espectrogramas\60Ig4PmU9NMqKGDMR8Um0Tc2KXB3_0_counting-fast.png
Intentando cargar el archivo: c:\Users\Vicen\OneDrive\Escritorio\Covid\DeteccionCovidRespiracion\CoronaHack-Respiratory-Sound-Dataset\data\train\20200413\CrlV0ddIqdbtxNnkIhtPS5dvoX42\counting-fast.wav
Espectrograma guardado en: espectrogramas\CrlV0ddIqdbtxNnkIhtPS5dvoX42_0_counting-fast.png
Intentando cargar el archivo: c:\

c:\Users\Vicen\anaconda3\envs\audio_env\lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


Espectrograma guardado en: espectrogramas\U1xH6cTpR6VhCLwJKx2wKZc5XE03_0_counting-fast.png
Intentando cargar el archivo: c:\Users\Vicen\OneDrive\Escritorio\Covid\DeteccionCovidRespiracion\CoronaHack-Respiratory-Sound-Dataset\data\train\20200417\yHRFkSwYEehBF4AtthYzXkR6jUc2\counting-fast.wav
Espectrograma guardado en: espectrogramas\yHRFkSwYEehBF4AtthYzXkR6jUc2_0_counting-fast.png
Intentando cargar el archivo: c:\Users\Vicen\OneDrive\Escritorio\Covid\DeteccionCovidRespiracion\CoronaHack-Respiratory-Sound-Dataset\data\train\20200417\uJEVN893b2Pntd1Ck3cdY7RZZRo1\counting-fast.wav
Espectrograma guardado en: espectrogramas\uJEVN893b2Pntd1Ck3cdY7RZZRo1_0_counting-fast.png
Intentando cargar el archivo: c:\Users\Vicen\OneDrive\Escritorio\Covid\DeteccionCovidRespiracion\CoronaHack-Respiratory-Sound-Dataset\data\train\20200417\RYJRtbTc26a12dQ5ddAwV2hJ4Cv1\counting-fast.wav
Espectrograma guardado en: espectrogramas\RYJRtbTc26a12dQ5ddAwV2hJ4Cv1_0_counting-fast.png
Intentando cargar el archivo: c:\